In [1]:
import json
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox
import random
import numpy as np

%matplotlib inline

## Load and Clean Data

In [2]:
process_folder = '../../process'
process_config_path = '../../process/configuration/adelaide.json'
pop_col = ["pop_ghs_2015"]
dest_col = ["destinations"]
np.random.seed(24)

In [3]:
with open(process_config_path) as json_file:
    config = json.load(json_file)

input_folder = os.path.join(process_folder, config['folder'])

gpkg_input = os.path.join(input_folder, config['geopackagePath'])

In [4]:
pop = gpd.read_file(gpkg_input, layer='pop_ghs_2015' )

In [5]:
pop.columns

Index(['index', 'pop_est', 'area_sqkm', 'pop_per_sqkm', 'intersection_count',
       'intersections_per_sqkm', 'count_pub', 'count_cafe', 'count_restaurant',
       'count_convenience', 'count_bar', 'count_fast_food', 'count_food_court',
       'count_fresh_food_market', 'count_pt_any', 'geometry'],
      dtype='object')

In [6]:
dests = gpd.read_file(gpkg_input, layer='destinations' )

In [7]:
dests.columns

Index(['dest_oid', 'osm_id', 'dest_name', 'dest_name_full', 'edge_ogc_fid',
       'n1', 'n2', 'n1_distance', 'n2_distance', 'match_point_distance',
       'geometry'],
      dtype='object')

In [8]:
fresh_food = dests[dests['dest_name_full'].str.contains('Fresh Food / Market')]

In [9]:
gdf_study_area = gpd.read_file(gpkg_input, layer="urban_study_region")
study_area = gdf_study_area["geometry"].iloc[0]

In [10]:
crs = gdf_study_area.crs
if pop.crs != crs:
    pop = pop.to_crs(crs)
if fresh_food.crs != crs:
    fresh_food = fresh_food.to_crs(crs)

In [11]:
import warnings

warnings.filterwarnings("ignore", "GeoSeries.notna", UserWarning)  # temp warning suppression
pop_clipped = gpd.clip(pop, study_area)
fresh_food_clipped = gpd.clip(fresh_food, study_area)

## Create Density-Based Quintiles

In [12]:
pop_clipped['pop_quintile'] = pd.qcut(pop_clipped['pop_est'], 5, labels=False)

## Assign Hexagon-Data to Destinations

In [13]:
joined_freshfood = gpd.sjoin(fresh_food_clipped, pop_clipped, how='left', op='within')

In [14]:
joined_freshfood.shape

(372, 28)

In [15]:
cleaned_joined_freshfood = joined_freshfood[~joined_freshfood['dest_name_full'].isnull()]

In [16]:
cleaned_joined_freshfood.columns

Index(['dest_oid', 'osm_id', 'dest_name', 'dest_name_full', 'edge_ogc_fid',
       'n1', 'n2', 'n1_distance', 'n2_distance', 'match_point_distance',
       'geometry', 'index_right', 'index', 'pop_est', 'area_sqkm',
       'pop_per_sqkm', 'intersection_count', 'intersections_per_sqkm',
       'count_pub', 'count_cafe', 'count_restaurant', 'count_convenience',
       'count_bar', 'count_fast_food', 'count_food_court',
       'count_fresh_food_market', 'count_pt_any', 'pop_quintile'],
      dtype='object')

## Select Destinations

In [17]:
q1_count_list = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 0].values.tolist()
q2_count_list = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 1].values.tolist()
q3_count_list = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 2].values.tolist()
q4_count_list = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 3].values.tolist()
q5_count_list = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 4].values.tolist()

In [18]:
count = 0
if len(q1_count_list) < 10:
    q1_sampledeficit = 10 - len(q1_count_list)
    if q1_sampledeficit % 4 == 0:
        count = count
    else:
        count = count + (q1_sampledeficit % 4)
    extra_q1 = (int(q1_sampledeficit/4))
    q1_dests = len(q1_count_list) 
else:
    extra_q1 = 0
    q1_dests = 10
if len(q2_count_list) < (extra_q1 + 10):
    q2_sampledeficit = (extra_q1 + 10) - len(q2_count_list)
    if q2_sampledeficit % 3 == 0:
        count = count
    else:
        count = count +(q2_sampledeficit % 3)
    extra_q2 = (int(q2_sampledeficit/3))
    q2_dests = len(q2_count_list)
else:
    extra_q2 = 0
    q2_dests = 10 + extra_q1
if len(q3_count_list) < (extra_q1 + extra_q2 + 10):
    q3_sampledeficit = (extra_q1 + extra_q2 + 10) - len(q3_count_list)
    if q3_sampledeficit % 2 == 0:
        count = count
    else:
        count = count + (q3_sampledeficit % 2)
    extra_q3 = (int(q3_sampledeficit/2))
    q3_dests = len(q3_count_list)
else:
    extra_q3 = 0
    q3_dests = 10 + extra_q1 + extra_q2
if len(q4_count_list) < (extra_q1 + extra_q2 + extra_q3 + 10):
    q4_sampledeficit = (extra_q1 + extra_q2 + extra_q3 + 10) - len(q4_count_list)
    extra_q4 = (int(q4_sampledeficit))
    q4_dests = len(q4_count_list)
else:
    extra_q4 = 0
    q4_dests = 10 + extra_q1 + extra_q2 + extra_q3
if len(q5_count_list) < (extra_q1 + extra_q2 + extra_q3 + extra_q4 + 10):
    print("fewer than 50 destinations exist")
else:
    q5_dests = 10 + extra_q1 + extra_q2 + extra_q3 + extra_q4 + count

In [19]:
q1_count = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 0]
q2_count = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 1]
q3_count = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 2]
q4_count = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 3]
q5_count = cleaned_joined_freshfood.loc[cleaned_joined_freshfood['pop_quintile'] == 4]

In [20]:
q1_sample_dests = q1_count.sample(q1_dests)
q2_sample_dests = q2_count.sample(q2_dests)
q3_sample_dests = q3_count.sample(q3_dests)
q4_sample_dests = q4_count.sample(q4_dests)
q5_sample_dests = q5_count.sample(q5_dests)

In [21]:
sample_dests = [q1_sample_dests, q2_sample_dests, q3_sample_dests, q4_sample_dests, q5_sample_dests]

In [22]:
final_sample_dests = pd.concat(sample_dests)

In [23]:
final_sample_dests = final_sample_dests.reset_index(drop=True)

In [24]:
final_sample_dests.head(1)

,dest_oid,osm_id,dest_name,dest_name_full,edge_ogc_fid,n1,n2,n1_distance,n2_distance,match_point_distance,...,count_pub,count_cafe,count_restaurant,count_convenience,count_bar,count_fast_food,count_food_court,count_fresh_food_market,count_pt_any,pop_quintile
0,23,5336040626,fresh_food_market,Fresh Food / Market,114871,5334832864,5334832855,56,168,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0


In [25]:
final_sample_dests = final_sample_dests.to_crs({'init': 'epsg:4326'})

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [26]:
final_sample_dests['lat'] = final_sample_dests.geometry.y
final_sample_dests['lon'] = final_sample_dests.geometry.x

In [27]:
final_sample_dests.head()

,dest_oid,osm_id,dest_name,dest_name_full,edge_ogc_fid,n1,n2,n1_distance,n2_distance,match_point_distance,...,count_restaurant,count_convenience,count_bar,count_fast_food,count_food_court,count_fresh_food_market,count_pt_any,pop_quintile,lat,lon
0,23,5336040626,fresh_food_market,Fresh Food / Market,114871,5334832864,5334832855,56,168,14,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,-35.115960,138.497700
1,389,552718759,fresh_food_market,Fresh Food / Market,208545,363242888,363242879,0,104,28,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,-35.113667,138.492695
2,466,26805247,fresh_food_market,Fresh Food / Market,42042,1069616575,1069616180,12,0,4,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,-34.790441,138.695935
3,190,7006866699,fresh_food_market,Fresh Food / Market,133757,1920840918,6010677658,123,8,23,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,-34.906512,138.578225
4,393,203684561,fresh_food_market,Fresh Food / Market,182971,632219160,2136833724,43,42,24,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,-34.881401,138.491495


In [28]:
cities = ['adelaide']
filenames = {}

for city in cities: 
    for destination in final_sample_dests:
        city_name = {city}
        hexagon_pop_quantile = final_sample_dests['pop_quintile']
        latitude = final_sample_dests['lat']
        longitude = final_sample_dests['lon']
        google_maps_screenshot = f"{latitude}_{longitude}_{city}_google_maps_image"
        google_satellite_screenshot = f"{latitude}_{longitude}_{city}_google_satellite_image"
        google_street_view_screenshot = f"{latitude}_{longitude}_{city}_google_street_view_image"

        # calculate total street length and edge count in each dataset, then add to indicators
        filenames[destination]["City_Name"] = city_name
                  

                  
        filenames[destination]["Hexagon_Pop_Quintile"] = hexagon_pop_quantile
        filenames[destination]["Latitude"] = latitude
        filenames[destination]["Longitude"] = longitude
        filenames[destination]["Google_Maps_Date"] = ""
        filenames[destination]["Google_Maps_Screenshot"] = google_maps_screenshot
        filenames[destination]["Google_Satellite_Date"] = ""
        filenames[destination]["Google_Satellite_Screenshot"] = google_satellite_screenshot
        filenames[destination]["Google_Street_View_Date"] = ""
        filenames[destination]["Google_Street_View_Screenshot"] = google_street_view_screenshot
        filenames[destination]["Assessment"] = ""
        print(ox.ts(), f"finshed names for {city}")

# turn indicators into a dataframe and save to disk
df_filenames = pd.DataFrame(filenames).T
df_filenames.to_csv(filenames_filepath, index=True, encoding="utf-8")
print(ox.ts(), f'all done, saved filenames to disk at "{filenames_filepath}"')

KeyError: 'dest_oid'